Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature

In [ ]:
def find_by_patterns(patterns, list_to_search):
    res = [x for x in list_to_search if re.search(patterns[0], x)] 
    if len(patterns) >1:
        for i in patterns[1:]:
            res = [x for x in res if re.search(i, x)] 
    return res

def plot_specific_columns(result_df, columns_to_plot):
    result_df_to_plot = result_df[columns_to_plot]
    all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_to_plot, 'lines+markers', swap_xy=True)
    plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', reversed_y = True, iplot_option = True)

def filtr_by_antipatterns(init_list: list, antipatterns: list):
    new_list = init_list.copy()
    droped_values = []
    for j in antipatterns:
        new_list = [i for i in new_list if j not in i ]
    for i in init_list:
        if i not in new_list:
            droped_values.append(i)
    print(f"Удаленные совпадения по антипаттерну: {droped_values}")
    return new_list

def plot_by_patterns(result_df, group_patterns, antipatterns=[]):
    if type(group_patterns[0]) == str:
        columns_to_plot = find_by_patterns(group_patterns, result_df.columns)
    else:
        columns_to_plot = []
        for i in group_patterns:
            this_column_to_plot = find_by_patterns(i, result_df.columns)
            columns_to_plot += this_column_to_plot
    print(f"Найденные совпадения: {columns_to_plot}")
    if len(antipatterns)>0:
        print('cj')
        columns_to_plot = filtr_by_antipatterns(columns_to_plot, antipatterns)
    plot_specific_columns(result_df, columns_to_plot)

Настройки расчета

In [ ]:
calc_options ={"step_lenth_in_calc_along_wellbore_m":24.35,
                "without_annulus_space":False}

Скв. 1Ф Исследование 4РМ

In [ ]:
pb_bar = 9 * 10**5
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45}

well_data = {"h_intake_mes_m" :1211,
"h_intake_vert_m" : 1211,
"h_bottomhole_mes_m" : 1760, #1756.8
"h_bottomhole_vert_m": 1760,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':156,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":240,
"fw_perc":25}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.975, 8.495, 9.44, 10.365, 10.902, 11.272, 12.085, 12.907, 13.785, 14.67, 15.55],
                                 'h_mes_survey_m': [0, 957, 1057, 1157, 1211, 1257, 1357, 1457, 1557, 1657, 1757]})

Скв. 2Ф Исследование 5РМ

In [ ]:

pb_bar = 9 * 10**5
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45}

well_data = {"h_intake_mes_m" :1205.5,
"h_intake_vert_m" : 1205.5,
"h_bottomhole_mes_m" : 1756.8, #1756.8
"h_bottomhole_vert_m": 1756.8,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar': 114.35, #114.35
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40,
"fw_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.9, 1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435],
                                 'h_mes_survey_m': [0, 105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]})

Скв. 3Ф 6РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45,
'gamma_wat':1.04}

well_data = {"h_intake_mes_m" :1100,
"h_intake_vert_m" : 1100,
"h_bottomhole_mes_m" : 1775, #1756.8
"h_bottomhole_vert_m": 1775,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':163.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":36/0.86,
"fw_perc":23}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.1, 3.05, 4.95, 6.51, 8.23, 9.91, 12.02, 14.14, 16.35],
                                 'h_mes_survey_m': [0, 376, 576, 776, 976, 1176, 1376, 1576, 1775]})

Скв. 4Ф 7РМ

In [ ]:
pb_bar = 9 * 10**5,
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45,
'gamma_wat':1.064}

well_data = {"h_intake_mes_m" :1099,
"h_intake_vert_m" : 1099,
"h_bottomhole_mes_m" : 1670, #1756.8
"h_bottomhole_vert_m": 1670,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':120,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40/0.86,
"fw_perc":6.2}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.4, 1.28, 2.29, 3.82, 5.11, 6.75, 8.23, 10.0, 11.72],
                                 'h_mes_survey_m': [0, 263, 463, 663, 863, 1063, 1263, 1463, 1663]})

Скв.2Ф Исследование 2Пл Памятно-Сасовское месторождение стр.93

In [ ]:
pb_bar = 16,7 * 10**5
mu_oil_cp = 10.05
fluid_data = {"rsb_m3m3" : 121.72,
"gamma_oil" : 0.83,
"gamma_gas" : 0.82}

well_data = {"h_intake_mes_m" :2519,
"h_intake_vert_m" : 2519,
"h_bottomhole_mes_m" : 2730, 
"h_bottomhole_vert_m": 2730,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":23.2,
"t_bottomhole_c" :76,
"t_earth_init_in_reservoir_c" : 76,
'p_bottomhole_bar':225,
"d_casing_inner_m" : 0.1553,
"d_tube_inner_m" : 0.076,
"qliq_on_surface_m3day":209.9,
"fw_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [5.73, 8.12, 11.0, 14.15, 17.40, 19.13, 21.41, None],
                                  't_survey_c': [23.2, 32.0, 43.5, 55.0, 66.0, 70.2, 75.6, 76.0],
                                 'h_mes_survey_m': [0, 500, 1000, 1500, 2000, 2250, 2585, 2700]})

In [ ]:
real_measurements["p_survey_bar"] = real_measurements["p_survey_mpa"] * 10
real_measurements = real_measurements.set_index(real_measurements['h_mes_survey_m'])
real_measurements.index.name = 'well.h_calculated_mes_m'
real_measurements

In [ ]:
pipe = Pipe.Pipe(temp_cor=uTemperature.temp_cor_simple_line.SimpleLineCor())


In [ ]:
pipe = Pipe.Pipe()


In [ ]:
fluid = BlackOil_model.Fluid(**fluid_data)
simple_well = self_flow_well.self_flow_well(fluid=fluid, pipe=pipe, **well_data, **calc_options)
simple_well.well_work_time_sec = 1

In [ ]:
simple_well.calc_all_from_down_to_up()

In [ ]:
result_df = simple_well.data.get_data_as_df()
result_df.to_excel('test.xlsx')
result_df = result_df.set_index('well.h_calculated_mes_m')
result_df.head()

In [ ]:
result_df = result_df.join(real_measurements, how = 'outer')

In [ ]:
for i in result_df.columns:
    print(i)

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey']]
antipatterns = ['mu', 'h_mes_survey', 'mpa']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["rho"]
antipatterns = ['cor_number', 'earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["mu"]
antipatterns = ['earth', 'cor_number']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["m3day"]
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["perc",'fw']
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["d_m"]
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["rs","m3m3"]
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["msec"]
antipatterns = ['secm', 'msecpam']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["pb"]
antipatterns = ['tpb', 'number']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
simple_well.data.print_all_names()

In [ ]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Расчет фонтанирующей скважины',  yaxis=dict(title = 'Измеренная глубина, м', autorange='reversed'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [ ]:
#wells = []
#dfs = []
#for i in range(1, 300, 50):
#    simple_well = self_flow_well.self_flow_well(h_intake_mes_m=999,h_intake_vert_m=999,
#                                            h_bottomhole_mes_m=1000,h_bottomhole_vert_m=1000,
#                                            t_earth_init_in_reservoir_c=90, t_bottomhole_c=92,
#                                            d_casing_inner_m=0.12,
#                                            d_tube_inner_m=0.062,
#                                            qliq_on_surface_m3day = 50,
#                                                p_bottomhole_bar=50
#                                       )
#    simple_well.pipe.fluid_flow.fl.rsb_m3m3 = 10 * i
#    simple_well.well_work_time_sec = 1 
#    simple_well.calc_all_from_down_to_up()
#    df = data.extract_data_from_object(simple_well, 'well', [i])
#    dfs.append(df)
#    wells.append(simple_well)

In [ ]:
#traces = []
#for simple_well in wells:
#    trace1 = trace(simple_well.data.get_values(24), simple_well.data.get_values(23),  "Давление, бар")
#    traces.append(trace1)
#data=traces
#plot()    

In [ ]:
trace1 = trace(simple_well.data.get_values(24), simple_well.data.get_values(23),  "Давление, бар")
trace2 = trace(simple_well.data.get_values(25), simple_well.data.get_values(23),  "Температура флюида, С")
trace3 = trace(simple_well.data.get_values(26), simple_well.data.get_values(23),  "Температура земли, С")
data=[trace1,trace2,trace3]
plot()

In [ ]:
trace1 = go.Scatter3d(
    x=simple_well.well_profile.x_displacement_m,
    y=simple_well.well_profile.x_displacement_m * 0,
    z=simple_well.well_profile.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m),
    y=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m) * 0,
    z=simple_well.well_profile.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')